In [1]:
import logging
import os

import wiki as w # changed wiki to include '[]'
import traceback
from tqdm import tqdm

In [2]:
print(w.__file__)

/home/manni/ner-s2s/word_embedding/wiki.py


In [3]:
#WIKIXML = '/home/manni/data/wiki/enwiki-latest-pages-articles-multistream1.xml-p1p41242.bz2'
WIKIXML = '/home/manni/data/wiki/en-28102022.bz2'

In [4]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [5]:
logging.basicConfig(format='[%(asctime)s] %(message)s', level=logging.INFO)
os.makedirs('data/', exist_ok=True)

In [ ]:
wiki_sentences = w.WikiSentences(WIKIXML, 'en',lower=True)

In [ ]:
for sent in wiki_sentences:
    print(sent[:100])
    break

In [ ]:
sv.save(wiki_sentences,'sentences')

In [6]:
sentences = sv.load('sentences')

In [ ]:
from multiprocessing import Pool
import workers
from collections import Counter

In [ ]:
WINDOW = 3
TOTAL = sentences.wiki.length

In [ ]:
#https://gist.github.com/nwjlyons/621fabfc0d4c1119b2ad338f615ce4ef
def chunks(generator, chunk_size):
    """Yield successive chunks from a generator"""
    chunk = []

    for item in generator:
        if len(chunk) >= chunk_size:
            yield chunk
            chunk = [item]
        else:
            chunk.append(item)

    if chunk:
        yield chunk

In [ ]:
results = list()
if __name__ == "__main__":
    with Pool(processes=40) as pool:
        results = list(pool.apply_async(workers.getTuples, args=(sent,WINDOW)) \
                       for sent in tqdm(sentences,position=0,total=TOTAL))
        results = [r.get() for r in results if r.get()]

In [ ]:
results = [item for sublist in results for item in sublist]

In [ ]:
c = Counter(results)
sv.save(dict(c),'PairFreqs')